In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [2]:
lats = np.random.uniform(low=-90, high=90, size=1500)
longs = np.random.uniform(low=-180, high=90, size=1500)
lat_longs = zip(lats, longs)
lat_longs

In [3]:
coordinates = list(lat_longs)

In [6]:
cities = list()

for coordinate in coordinates:
    city = citipy.nearest_city(latitude=coordinate[0], longitude=coordinate[1])
    if city not in cities:
        cities.append(city)
        
len(cities)

566